In [1]:
import brightway2 as bw
import os
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
bw.projects.set_current('testowm')
bw.databases

Databases dictionary with 0 objects

In [3]:
import bw2io as bi
bw.bw2setup()
bw.databases

Creating default biosphere

Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applying strategy: ensure_categories_are_tuples
Applied 3 strategies in 0.00 seconds
Title: Writing activities to SQLite3 database:
  Started: 06/09/2025 12:45:30
  Finished: 06/09/2025 12:45:30
  Total time elapsed: 00:00:00
  CPU %: 99.40
  Memory %: 1.11
Created database: biosphere3
Creating default LCIA methods

Wrote 762 LCIA methods with 227223 characterization factors
Creating core data migrations



Databases dictionary with 1 object(s):
	biosphere3

In [4]:
# another way
bw.import_ecoinvent_release('3.8', 'cutoff','ebenezer.kwofie@mcgill.ca', '2EBz*!#0DCH4')

Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applying strategy: ensure_categories_are_tuples
Applied 3 strategies in 0.00 seconds
4421 datasets
0 exchanges
0 unlinked exchanges
  
Title: Writing activities to SQLite3 database:
  Started: 06/09/2025 12:45:57
  Finished: 06/09/2025 12:45:57
  Total time elapsed: 00:00:00
  CPU %: 98.80
  Memory %: 2.99
Created database: ecoinvent-3.8-biosphere
Extracting XML data from 19565 datasets
Extracted 19565 datasets in 28.16 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_eco

In [5]:
imp = bw.ExcelImporter(r"Canada OWM Facilities Database.xlsx") # the path to your inventory excel file
imp.apply_strategies()
imp.match_database("ecoinvent-3.8-cutoff", fields=('name', 'unit', 'location', 'reference product')) # 'reference product'
imp.match_database(fields=('name', 'unit', 'location'))
imp.statistics()
imp.write_excel(only_unlinked=True)
list(imp.unlinked)
imp.write_database()

Extracted 3 worksheets in 0.05 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 1.93 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
16 datasets
140 exchanges
0 unlinked exchanges
  
Wrote matching file to:
/Users/yonata

In [6]:
imp = bw.ExcelImporter(r"Scenarios Database.xlsx") # the path to your inventory excel file
imp.apply_strategies()
imp.match_database("OWM LCA", fields=('name', 'unit', 'location', 'reference product')) # 'reference product'
imp.match_database(fields=('name', 'unit', 'location'))
imp.statistics()
imp.write_excel(only_unlinked=True)
list(imp.unlinked)
imp.write_database()

Extracted 1 worksheets in 0.02 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 1.96 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
3 datasets
26 exchanges
0 unlinked exchanges
  
Wrote matching file to:
/Users/yonatanb

In [7]:
#check if it imported properly
list(bw.databases)

['biosphere3',
 'ecoinvent-3.8-biosphere',
 'ecoinvent-3.8-cutoff',
 'OWM LCA',
 'Scenarios']

In [8]:
LCAdb =  bw.Database('Scenarios')
S1 = [a for a in LCAdb if 'S1' in a ['name']]
S2 = [a for a in LCAdb if 'S2' in a ['name']]
S3 = [a for a in LCAdb if 'S3' in a ['name']]
acts = S1[0], S2[0], S3[0]
acts

('S1' (tonne, CA-QC, None),
 'S2' (tonne, CA-QC, None),
 'S3' (tonne, CA-QC, None))

In [9]:
CC_method = [m for m in bw.methods if 'IPCC 2013' in str(m) and not 'LT' in str(m) and 'GWP 100' in str(m)]
CC_method

[('IPCC 2013', 'climate change', 'GWP 100a')]

In [10]:
FU = [{x:1} for x in acts] #functional unit
bw.calculation_setups['OWM_Scenarios'] = {'inv':FU, 'ia': CC_method}
mylca = bw.MultiLCA('OWM_Scenarios')
mylca.results

array([[745.03337811],
       [558.47728048],
       [365.23302694]])